<a href="https://colab.research.google.com/github/hmtrii/tirg/blob/main/Compute_recall.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# INSTALL

In [ ]:
!git clone https://github.com/hmtrii/tirg.git

In [ ]:
!rm -rf /content/Fashion200k
!mkdir /content/Fashion200k
!cp -r /content/drive/MyDrive/TIRG/dataset/fashion-200k/labels /content/Fashion200k
!cp /content/drive/MyDrive/TIRG/dataset/test_queries.txt /content/Fashion200k
!mkdir /content/Fashion200k/women

In [ ]:
!tar -xf /content/drive/MyDrive/TIRG/dataset/fashion-200k/women.tar.gz -C /content/Fashion200k/women

In [ ]:
!pip install tensorboardX

In [ ]:
!pip install torch==1.2.0 torchvision==0.4.0    

In [ ]:
!pip install Pillow==5.2.0

# LOAD DATASET

In [ ]:
cd /content/tirg

In [ ]:
from tqdm import tqdm
import torch
import numpy as np
from tools import pkl
import time
from test_retrieval import test
from main import load_dataset, create_model_and_optimizer
from LSH import create_hash_table

In [ ]:
class Opt:
    def __init__(self):
        self.dataset = "fashion200k"
        self.dataset_path = "/content/Fashion200k"
        self.batch_size = 32
        self.embed_dim = 512
        self.hashing = True
        self.retrieve_by_random = True
        self.model = "tirg"
        self.learning_rate = 1e-2
        self.weight_decay = 1e-6
        self.pretrained_model = "/content/drive/MyDrive/TIRG/runs/exp3/best_checkpoint.pth"

In [ ]:
opt = Opt()
trainset, testset = load_dataset(opt)
model, _ = create_model_and_optimizer(opt, [t for t in trainset.get_all_texts()])
model.eval()

Compute all images feature

In [ ]:
all_imgs = []
imgs = []
for i in tqdm(range(len(testset.imgs))):
    imgs += [testset.get_img(i)]
    if len(imgs) >= opt.batch_size or i == len(testset.imgs) - 1:
        if 'torch' not in str(type(imgs[0])):
            imgs = [torch.from_numpy(d).float() for d in imgs]
        imgs = torch.stack(imgs).float()

        if torch.cuda.is_available():
            imgs = torch.autograd.Variable(imgs).cuda()
        else:
            imgs = torch.autograd.Variable(imgs).cpu()

        imgs = model.extract_img_feature(imgs).data.cpu().numpy()
        all_imgs += [imgs]
        imgs = []
all_imgs = np.concatenate(all_imgs)
all_captions = [img['captions'][0] for img in testset.imgs]

In [ ]:
pkl.pkl_save("/content/all_captions.pkl", all_captions)
pkl.pkl_save("/content/all_imgs.pkl", all_imgs)

In [ ]:
def save_normalize_all_imgs_feature():
    all_imgs_feature = pkl.pkl_load("/content/all_imgs.pkl")
    for i in tqdm(range(all_imgs_feature.shape[0])):
        all_imgs_feature /= np.linalg.norm(all_imgs_feature[i,:])

    pkl.pkl_save("/content/all_norm_imgs_feature.pkl", all_imgs_feature)

save_normalize_all_imgs_feature()

Compute all queries feature

In [ ]:
imgs = []
mods = []
all_queries = []
test_queries = testset.get_test_queries()
for t in tqdm(test_queries):
    imgs += [testset.get_img(t['source_img_id'])]
    mods += [t['mod']['str']]
    if len(imgs) >= opt.batch_size or t is test_queries[-1]:
        if 'torch' not in str(type(imgs[0])):
            imgs = [torch.from_numpy(d).float() for d in imgs]
        imgs = torch.stack(imgs).float()

        if torch.cuda.is_available():
            imgs = torch.autograd.Variable(imgs).cuda()
        else:
            imgs = torch.autograd.Variable(imgs).cpu()

        mods = [t for t in mods]
        f = model.compose_img_text(imgs, mods).data.cpu().numpy()
        all_queries += [f]
        imgs = []
        mods = []
all_queries = np.concatenate(all_queries)
all_target_captions = [t['target_caption'] for t in test_queries]

In [ ]:
pkl.pkl_save("/content/all_queries.pkl", all_queries)
pkl.pkl_save("/content/all_target_captions.pkl", all_target_captions)

In [ ]:
def save_normalize_all_queries_feature():
    all_imgs_feature = pkl.pkl_load("/content/all_queries.pkl")
    for i in tqdm(range(all_imgs_feature.shape[0])):
        all_imgs_feature /= np.linalg.norm(all_imgs_feature[i,:])

    pkl.pkl_save("/content/all_norm_queries_feature.pkl", all_imgs_feature)
save_normalize_all_queries_feature()

Use pre-computed

In [ ]:
cp /content/*.pkl -d /content/drive/MyDrive/TIRG/pkl_ver3

In [ ]:
test_queries = testset.get_test_queries()
all_queries = pkl.pkl_load("/content/drive/MyDrive/TIRG/pkl_ver3/all_norm_queries_feature.pkl")
all_target_captions = pkl.pkl_load("/content/drive/MyDrive/TIRG/pkl_ver3/all_target_captions.pkl")
all_imgs = pkl.pkl_load("/content/drive/MyDrive/TIRG/pkl_ver3/all_norm_imgs_feature.pkl")
all_captions = pkl.pkl_load("/content/drive/MyDrive/TIRG/pkl_ver3/all_captions.pkl")

In [ ]:
# Create random queries
np.random.seed(0)
id_random = np.random.choice(all_queries.shape[0], 5000)
queries_feature = []
for id in id_random:
    queries_feature += [all_queries[id]]

queries_feature = np.array(queries_feature)

# Normal

In [ ]:
tic = time.time()
sims = queries_feature.dot(all_imgs.T)
for i, t in enumerate(test_queries):
	try:
		sims[i, t["source_img_id"]] = -10e10  # remove query image
	except:
		pass

nn_result = [np.argsort(-sims[i, :])[:110] for i in range(sims.shape[0])]
out = []
nn_result = [[all_captions[nn] for nn in nns] for nns in nn_result]
for k in [1, 5, 10, 50, 100]:
# for k in [100]:
    r = 0.0
    # for i, nns in enumerate(nn_result):
    for id, nns in zip(id_random, nn_result):
        if all_target_captions[id] in nns[:k]:
            r += 1
    r /= len(nn_result)
    out += [('recall_top' + str(k) + '_correct_composition', r)]
tac = time.time()
print(out)
print(f"{round(tac - tic, 3)}s")

# Hashing

In [ ]:
def compute_similary_by_hasing(all_imgs_feature, queries_feature, test_queries):
	hash_table = create_hash_table(all_imgs_feature, 12, 512)
	sims = []
	for i, query in enumerate(queries_feature):
		idx_imgs = hash_table.__getitem__(query)
		spatial_search = np.array([all_imgs_feature[i] for i in idx_imgs])
		if spatial_search.shape[0] != 0:
			dis_value = query.dot(spatial_search.T)
			sim = np.full(all_imgs_feature.shape[0], -10e10)
			for i in idx_imgs:
				sim[i] = dis_value[0]
				dis_value = dis_value[1:]
		else:
			sim = query.dot(all_imgs_feature.T)

		sim[test_queries[i]['source_img_id']] = -10e10
		sims += [sim]

	return np.array(sims)

In [ ]:
tic = time.time()
sims = compute_similary_by_hasing(all_imgs, queries_feature, test_queries)
nn_result = [np.argsort(-sims[i, :])[:100] for i in range(sims.shape[0])]
out = []
nn_result = [[all_captions[nn] for nn in nns] for nns in nn_result]
for k in [1, 5, 10, 50, 100]:
# for k in [100]:
    r = 0.0
# for i, nns in enumerate(nn_result):
    for id, nns in zip(id_random, nn_result):
        if all_target_captions[id] in nns[:k]:
            r += 1
    r /= len(nn_result)
    out += [('recall_top' + str(k) + '_correct_composition', r)]
tac = time.time()
print(out)
print(f"{round(tac - tic, 3)}s")